In [7]:
from PIL import Image
import numpy as np

import os
import scipy.io as io
import scipy
import scipy.ndimage
import matplotlib.pyplot as plt
import pandas as pd
import csv
from shutil import copyfile


In [8]:
base_path = 'D:\\OneDrive\\OneDrive - UvA\\ThesisData\\Datasets\\Shanghaitech\\SHTB_datasets\\SHTB_SIG_3.5\\train\\images'
sigma = 3.5

In [9]:
def generate_density(img, mat, sigma):
    """ Given an image, its corresponding ground-truth annotated file, and a standard deviation, creates the ground-truth density map.
        img: The image corresponding to the ground-truth annotations
        mat: The coordinates of the annotations.
        sigma: A constant sigma used for the gaussian filter."""
    
    w, h = img.size
    k = np.zeros((h, w))
    
    gt_points = mat["image_info"][0,0][0,0][0]
    for (x, y) in gt_points.astype(int):
        if x < w and y < h:
            k[y, x] = 1  # Note the order of x and y here. Height is stored in first dimension
        else:
            print("This should never happen!")  # This would mean a head is annotated outside the image.
    
    density = scipy.ndimage.filters.gaussian_filter(k, sigma , mode='constant' , truncate=2) #
    return density

In [10]:
def gen_all_densities(source_path, sigma):
    for i, img_path in enumerate(os.listdir(source_path)):
        print(i)
        
        s_img_path = os.path.join(source_path, img_path)
        s_gt_path = s_img_path.replace('images', 'gt').replace('IMG_', 'GT_IMG_').replace('.jpg', '.mat')
        
        t_img_path = s_img_path.replace('images', 'img').replace('IMG_', '')
        t_den_path = t_img_path.replace('img', 'den').replace('.jpg', '.csv')

        img = Image.open(s_img_path)
        w, h = img.size
        
        mat = io.loadmat(s_gt_path)
        
        den = generate_density(img, mat, sigma)
        den = den.astype(np.float32)
        
        copyfile(s_img_path, t_img_path)
        with open(t_den_path, 'w', newline='') as f:
            writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            for line in den:
                writer.writerow(line)


In [11]:
gen_all_densities(base_path, sigma)

0
1
2
3
4
5


KeyboardInterrupt: 

In [6]:
# den_path = 'D:\\OneDrive\\OneDrive - UvA\\ThesisData\\Datasets\\Shanghaitech\\SHTB_STD\\train\\den\\1.csv'
# den = pd.read_csv(den_path, header=None).values
# den = den.astype(np.float32, copy=False)